In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [ ]:
import pprint

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

In [ ]:
# import packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import sys
import csv
import gzip
import copy
import datetime
import pickle
from sklearn import metrics
from tabulate import tabulate

In [ ]:
seed_value = 42  # seed for reproducibility
random.seed(seed_value)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
sys.path.append('/content/drive/MyDrive/ctr/code/model')

In [ ]:
import dcn
import run_models

In [ ]:
BATCH_SIZE = 100000
SHUFFLE_BUFFER_SIZE = 10000
SHUFFLE_SEED = 42
DATASET_EPOCHS = 1
FITTING_EPOCHS = 50
LEARNING_RATE = 0.0001
DCN_PARALLEL = True
STR_COLUMNS = [
    'click', 'banner_pos',
    'site_id', 'site_domain', 'site_category',
    'app_id', 'app_domain', 'app_category',
    'device_id', 'device_ip', 'device_model', 'device_type', 'device_conn_type',
    'C1', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21',
    'day_of_week'
    ]
INT_COLUMNS = [
    'hour_of_day'
]
COLUMN_DEFAULTS = [tf.string] * (len(STR_COLUMNS) -1) + [tf.float32] * len(INT_COLUMNS) + [tf.string] * 1

In [ ]:
# column_defaults를 원본 CSV File의 Column 순서로 인식한다

In [ ]:
TRAIN_FILE = '/content/drive/MyDrive/ctr/avazu/processed/train/train.csv'
train_batches = tf.data.experimental.make_csv_dataset(
    TRAIN_FILE,
    batch_size=BATCH_SIZE,
    select_columns= STR_COLUMNS + INT_COLUMNS,
    column_defaults=COLUMN_DEFAULTS,
    shuffle=True, shuffle_buffer_size=SHUFFLE_BUFFER_SIZE, shuffle_seed=SHUFFLE_SEED,
    num_epochs=DATASET_EPOCHS
)

In [ ]:
TEST_FILE = '/content/drive/MyDrive/ctr/avazu/processed/train/test.csv'
test = tf.data.experimental.make_csv_dataset(
    TEST_FILE,
    batch_size=BATCH_SIZE,
    select_columns= STR_COLUMNS + INT_COLUMNS,
    column_defaults=COLUMN_DEFAULTS,
    num_epochs=DATASET_EPOCHS
)

In [ ]:
with open('/content/drive/MyDrive/ctr/avazu/processed/train/vocabularies.p', 'rb') as f:
    total_voca = pickle.load(f)

In [ ]:
FEATURE_ENGINEERING = {
    'embedding' : [
        'site_id', 'site_domain', 'site_category',
        'app_id', 'app_domain', 'app_category',
        'device_model', 'device_type',
        'C1', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21',
        'day_of_week'
    ]
}

In [ ]:
vocabularies = {}

vocabularies['embedding'] = {}
for feature in FEATURE_ENGINEERING['embedding']:
    vocabularies['embedding'][feature] = total_voca[feature]

In [ ]:
str_features = list(vocabularies['embedding'].keys())
int_features = ['hour_of_day']

In [ ]:
nodes = [32, 64, 128, 256, 512, 1024]
deep_layers = [1, 2, 3, 4, 5]
cross_layers = [1, 2, 3, 4, 5, 6]
result_dict = {}

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
for engineering_method, voca_set in vocabularies.items():
    for feature, voca in voca_set.items():
        vocabularies[engineering_method][feature] = list(map(str, voca))

In [ ]:
HISTORY_FILE_DIR = '/content/drive/MyDrive/ctr/data/baseline/'
HISTORY_FILE_NAME = 'fitting_history.V1.0'

In [ ]:
CHECKPOINTS_DIR = '/content/drive/MyDrive/ctr/data/baseline/'
CHECKPOINTS_NAME = 'checkpoints.V1.0'

In [ ]:
node, deep_layer, cross_layer = 1024, 4, 1
print("Node: {}, Deep Layer : {}, Cross Layer : {}".format(node, deep_layer, cross_layer))

fitting_history = run_models.run_models(
    dcn_parallel=DCN_PARALLEL,
    cross_layer_size=cross_layer,
    deep_layer_sizes=[node]*deep_layer,
    vocabularies=vocabularies,
    str_features=str_features,
    int_features=int_features,
    train=train_batches,
    test=test,
    learning_rate=LEARNING_RATE,
    epochs=FITTING_EPOCHS,
    history_file_dir=HISTORY_FILE_DIR,
    history_file_name=HISTORY_FILE_NAME,
    checkpoints_dir=CHECKPOINTS_DIR,
    checkpoints_name=CHECKPOINTS_NAME
    )

Node: 1024, Deep Layer : 4, Cross Layer : 1
363/363 ━━━━━━━━━━━━━━━━━━━━ 944s 3s/step - AUC: 0.5942 - LogLoss: 0.5121 - loss: 0.4895 - regularization_loss: 0.0000e+00 - total_loss: 0.4895
43/43 ━━━━━━━━━━━━━━━━━━━━ 112s 3s/step - AUC: 0.6812 - LogLoss: 0.4412 - loss: 0.4721 - regularization_loss: 0.0000e+00 - total_loss: 0.4721
1th Epoch
363/363 ━━━━━━━━━━━━━━━━━━━━ 1002s 3s/step - AUC: 0.6875 - LogLoss: 0.4307 - loss: 0.4257 - regularization_loss: 0.0000e+00 - total_loss: 0.4257
43/43 ━━━━━━━━━━━━━━━━━━━━ 126s 3s/step - AUC: 0.7173 - LogLoss: 0.4256 - loss: 0.4500 - regularization_loss: 0.0000e+00 - total_loss: 0.4500
363/363 ━━━━━━━━━━━━━━━━━━━━ 1099s 3s/step - AUC: 0.7271 - LogLoss: 0.4138 - loss: 0.4117 - regularization_loss: 0.0000e+00 - total_loss: 0.4117
43/43 ━━━━━━━━━━━━━━━━━━━━ 130s 3s/step - AUC: 0.7278 - LogLoss: 0.4050 - loss: 0.4258 - regularization_loss: 0.0000e+00 - total_loss: 0.4258
363/363 ━━━━━━━━━━━━━━━━━━━━ 1111s 3s/step - AUC: 0.7269 - LogLoss: 0.4112 - loss: 0.4

In [ ]:
HISTORY_FILE_PATH = '/content/drive/MyDrive/ctr/data/baseline/fitting_history.V1.0.p'

with open(HISTORY_FILE_PATH, 'wb') as f:
    pickle.dump(fitting_history, f)